To be run on perlmutter

In [1]:
import os
import sys
sys.path.insert(0, "/global/cfs/cdirs/m4129/projects/crispy_shifty_adam")

In [2]:
from crispy_shifty.utils.io import fix_path_prefixes

pairs = '/global/u2/b/broerman/crispy_shifty/projects/crispy_shifty_dimers/round_3/design/04_two_state/mpnn_paired_states.pair'
new_pairs = fix_path_prefixes(
    find="/home",
    replace="/global/u2/b",
    file=pairs,
    overwrite=True,
)

/global/cfs/cdirs/m4129/projects/crispy_shifty_adam/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [4]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "05_fold_Y"
design_list_file = '/global/u2/b/broerman/crispy_shifty/projects/crispy_shifty_dimers/round_3/design/04_two_state/mpnn_paired_states.pair'
output_path = f"/pscratch/sd/b/broerman/{simulation_name}"

options = " ".join(
    [
        "out:level 200",
    ]
)
extra_kwargs = {"models": "all"}

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.folding.fold_dimer_Y",
    design_list_file=design_list_file,
    output_path=output_path,
    perlmutter_mode=True,
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
    time="45:00",
)

Run the following command with your desired environment active:
sbatch -a 1-3837 /pscratch/sd/b/broerman/05_fold_Y/run.sh


In [5]:
import socket
print(f"ssh {socket.gethostname()}")

ssh nid002845


In [7]:
print("""module load cudatoolkit/11.4
N=$(( SLURM_ARRAY_TASK_ID - 1 ))
N2=$(( N + 1 ))
start_idx=$(( N*4 + 1 ))
end_idx=$(( N2*4 ))
source activate /global/cfs/cdirs/m4129/projects/crispy_shifty_adam/envs/crispy
time head -n $end_idx /pscratch/sd/b/broerman/05_fold_Y/tasks.cmds | tail -n +$start_idx | parallel 'CUDA_VISIBLE_DEVICES=$(("{%}" - 1)) && bash -c {}'""")

module load cudatoolkit/11.4
N=$(( SLURM_ARRAY_TASK_ID - 1 ))
N2=$(( N + 1 ))
start_idx=$(( N*4 + 1 ))
end_idx=$(( N2*4 ))
source activate /global/cfs/cdirs/m4129/projects/crispy_shifty_adam/envs/crispy
time head -n $end_idx /pscratch/sd/b/broerman/05_fold_Y/tasks.cmds | tail -n +$start_idx | parallel 'CUDA_VISIBLE_DEVICES=$(("{%}" - 1)) && bash -c {}'
